💬 Day 1 Challenge: Build a Benchmark / Index
🎯 Challenge Statement

Create a Python script or notebook that builds a blended, index-like benchmark that can later be used to compare any investment portfolio’s performance in terms of risk and return.

Your goal is to fetch suitable benchmarks for each of the asset classes in your portfolio from yfinance and ensure data for at least 10 years is available for all of them.

Then, establish and assign target weights to each of the benchmark elements. The sum of the weights must be 1.

🧠 Objectives

By the end of today’s challenge, you should have:

A selection of tickers, representing the components of the benchmark (benchmark_tickers).

Weights to be attached to these benchmark components (benchmark_weights).

A dataset with historical prices for the components of the benchmark, in the form of a reusable DataFrame, ready for Day 2’s portfolio comparison.

In [1]:
# Make sure yfinance is up-to-date
!python -m pip install --upgrade yfinance pandas pyarrow

In [2]:
# --- Setup imports ---
import sys, os
sys.path.append(os.path.abspath(".."))
from portfolio_analyzer.build_benchmark import (
    collect_metadata, save_metadata,
    get_benchmark_data_with_names, save_benchmark_data
)

In [20]:
# Input index/ETF tickers for each market segment (yfinance-compatible)
import os
import json

benchmark_tickers = {
    "international_equity": ["EFA"],         # iShares MSCI EAFE ETF, Asset Class: International Developed Markets Equity
    "us_small_cap": ["IWM"],         # iShares Russell 2000 ETF, Asset Class: U.S. Small-Cap Equity
    "us_mid_cap": ["IJH"],         # iShares Core S&P Mid-Cap ETF, Asset Class: U.S. Mid-Cap Equity
    "us_large_cap": ["SPY"],         # SPDR S&P 500 ETF, Asset Class: U.S. Large-Cap Equity
    "us_fixed_income": ["AGG"],            # iShares Core U.S. Aggregate Bond ETF, Asset Class: U.S. Investment-Grade Bonds (Fixed Income)
    "liquidity": ["BIL"]         # 1–3 Month T-Bill ETF (cash proxy), Asset Class: U.S. Treasury Bills (Ultra-Short-Term Fixed Income)
}

# Save benchmarks to json for future reference
# Ensure the data directory exists
os.makedirs("../data", exist_ok=True)

with open("../data/benchmark_tickers.json", "w") as f:
    json.dump(benchmark_tickers, f, indent=4)

In [5]:
# --- Run metadata collection ---
metadata_df = collect_metadata(benchmark_tickers)
save_metadata(metadata_df)

✅ Metadata saved to ../data\benchmark_metadata.json


In [6]:
# --- Fetch price data ---
benchmark_prices_named = get_benchmark_data_with_names(
    benchmark_tickers,
    start="2015-11-08",
    end="2025-11-07"
)
save_benchmark_data(benchmark_prices_named)

✅ Benchmark data: 2514 days × 6 tickers
   From 2015-11-09 to 2025-11-06
✅ Saved CSV and Parquet data to ../data


In [24]:
# Establish portfolio weights for benchmarks
import json
from tabulate import tabulate

# Asset class weights
equity_weight = 0.74
fixed_income_weight = 0.21
liquidity_weight = 0.05

# Breakdown of equity allocation
domestic_equity_weight = 0.85 * equity_weight
international_equity_weight = 0.15 * equity_weight
us_large_cap_weight = 0.50 * domestic_equity_weight
us_mid_cap_weight = 0.35 * domestic_equity_weight
us_small_cap_weight = 0.15 * domestic_equity_weight

# Verify that weights sum to 1.0
sum_of_weights = (
    us_large_cap_weight +
    us_mid_cap_weight +
    us_small_cap_weight +
    international_equity_weight +
    fixed_income_weight +
    liquidity_weight)

# Defined portfolio weights for each benchmark (segment)
benchmark_weights = {
    "international_equity": international_equity_weight,
    "us_small_cap": us_small_cap_weight,
    "us_mid_cap": us_mid_cap_weight,
    "us_large_cap": us_large_cap_weight,
    "us_fixed_income": fixed_income_weight,
    "liquidity": liquidity_weight
}   

# Sum of weights should equal 1.0
assert abs(sum(benchmark_weights.values()) - 1.0) < 1e-6, "Benchmark weights must sum to 1.0"

# Convert the dictionary to a list of tuples
weights_table = [(k.replace("_", " ").title(), v) for k, v in benchmark_weights.items()]

# Add a blank row, summary row
weights_table.append(("", ""))  # ← this creates the visual blank line
weights_table.append(("Sum of Weights", sum(benchmark_weights.values())))

# Display the table
print(tabulate(weights_table, headers=["Segment", "Weight"], tablefmt="github", floatfmt=".4f"))

# Save benchmark weights to json for future reference
with open("../data/benchmark_weights.json", "w") as f:
    json.dump(benchmark_weights, f, indent=4)

| Segment              | Weight              |
|----------------------|---------------------|
| International Equity | 0.111               |
| Us Small Cap         | 0.09435             |
| Us Mid Cap           | 0.22014999999999998 |
| Us Large Cap         | 0.3145              |
| Us Fixed Income      | 0.21                |
| Liquidity            | 0.05                |
|                      |                     |
| Sum of Weights       | 1.0                 |


In [10]:
# Display the compiled metadata
from tabulate import tabulate

cols = ["segment", "ticker", "name", "fund_family", "inception_date"]
print(tabulate(metadata_df[cols].values, headers=cols, tablefmt="github"))

| segment              | ticker   | name                                 | fund_family                             | inception_date   |
|----------------------|----------|--------------------------------------|-----------------------------------------|------------------|
| international_equity | EFA      | iShares MSCI EAFE ETF                | iShares                                 | 2001-08-14       |
| us_small_cap         | IWM      | iShares Russell 2000 ETF             | iShares                                 | 2000-05-22       |
| us_mid_cap           | IJH      | iShares Core S&P Mid-Cap ETF         | iShares                                 | 2000-05-22       |
| us_large_cap         | SPY      | SPDR S&P 500 ETF                     | SPDR State Street Investment Management | 1993-01-22       |
| us_fixed_income      | AGG      | iShares Core U.S. Aggregate Bond ETF | iShares                                 | 2003-09-22       |
| liquidity            | BIL      | SPDR Bloombe

So, the shortest history available is back to May, 2007 for our liquidity benchmark. That's about 18 years of history, which is plenty. The other benchmarks have even more available history.

In [11]:
# Function to view a small sample of randomly selected data (name, ticker, close) for each benchmark

def inspect_random_price_segment(benchmark_tickers, start="2015-01-01", end=None,
                                 window_min=3, window_max=5, seed=None):
    """
    Randomly selects 10–20 consecutive days of adjusted 'Close' prices
    for each ticker, for inspection.

    With auto_adjust=True, all OHLC data are adjusted for splits/dividends,
    so 'Close' already reflects total return equivalence.
    """
    if seed is not None:
        np.random.seed(seed)

    # Flatten ticker list from dictionary values
    tickers = [t for sublist in benchmark_tickers.values() for t in sublist]

    # Auto-adjust so 'Close' is total-return adjusted
    data = yf.download(
        tickers=tickers,
        start=start,
        end=end,
        interval="1d",
        auto_adjust=True,
        progress=False
    )

    # Extract only auto-adjusted closing prices
    if isinstance(data.columns, pd.MultiIndex):
        close_data = data["Close"]
    else:
        # Single ticker case
        close_data = data[["Close"]]

    samples = {}
    for ticker in tickers:
        # Drop missing rows
        df = close_data[ticker].dropna().to_frame(name="Close")
        n = len(df)
        if n < window_min:
            print(f"⚠️ Not enough data for {ticker} ({n} rows). Skipping.")
            continue

        # Random window
        window_len = np.random.randint(window_min, window_max + 1)
        start_idx = np.random.randint(0, n - window_len)
        sample = df.iloc[start_idx : start_idx + window_len].copy()

        samples[ticker] = sample
        print(f"📈 {ticker}: Showing {window_len} adjusted days starting {sample.index[0].date()}")
        print(sample)
        print("-" * 80)

    return samples

In [16]:
# Implement the function to inspect some closing prices for each benchmark
import yfinance as yf
import pandas as pd
import numpy as np

samples = inspect_random_price_segment(
    benchmark_tickers,
    start="2015-01-01",
    window_min=3,
    window_max=5,
    seed=42
)

📈 EFA: Showing 5 adjusted days starting 2018-06-04
                Close
Date                 
2018-06-04  56.115337
2018-06-05  56.027420
2018-06-06  56.530949
2018-06-07  56.211246
2018-06-08  56.275196
--------------------------------------------------------------------------------
📈 IWM: Showing 5 adjusted days starting 2019-07-01
                 Close
Date                  
2019-07-01  144.178833
2019-07-02  143.301300
2019-07-03  144.326569
2019-07-05  144.714508
2019-07-08  143.338318
--------------------------------------------------------------------------------
📈 IJH: Showing 3 adjusted days starting 2021-07-07
                Close
Date                 
2021-07-07  50.296219
2021-07-08  49.670357
2021-07-09  50.717220
--------------------------------------------------------------------------------
📈 SPY: Showing 4 adjusted days starting 2016-11-07
                 Close
Date                  
2016-11-07  183.992950
2016-11-08  184.821671
2016-11-09  186.781128
2016-11-10  1

The data appear to be fine, so now let's create a dataframe for use in tomorrow's Daily Python Challenge.

In [ ]:
# Fetch benchmark prices with fund names as columns
benchmark_prices_named = get_benchmark_data_with_names(
    benchmark_tickers,
    start="2015-11-08",
    end="2025-11-07"
)

✅ Benchmark data: 2514 days × 6 tickers
   From 2015-11-09 to 2025-11-06


In [ ]:
# Display the first few rows to inspect the data
benchmark_prices_named.head()

Ticker,iShares Core U.S. Aggregate Bond ETF,SPDR Bloomberg 1-3 Month T-Bill ETF,iShares MSCI EAFE ETF,iShares Core S&P Mid-Cap ETF,iShares Russell 2000 ETF,SPDR S&P 500 ETF
Date,,,,,,
2015-11-09,82.358589,75.228943,45.124557,24.905758,103.394669,175.830460
2015-11-10,82.510834,75.212471,45.094624,25.046715,103.701836,176.236038
2015-11-11,82.404198,75.228943,45.326572,24.898882,102.797913,175.543198
2015-11-12,82.480408,75.228943,44.735485,24.362566,100.832199,173.092606
2015-11-13,82.640320,75.212471,44.361382,24.151144,100.103790,171.149094


In [ ]:
# Display the last few rows to inspect the data
benchmark_prices_named.tail()

Ticker,iShares Core U.S. Aggregate Bond ETF,SPDR Bloomberg 1-3 Month T-Bill ETF,iShares MSCI EAFE ETF,iShares Core S&P Mid-Cap ETF,iShares Russell 2000 ETF,SPDR S&P 500 ETF
Date,,,,,,
2025-10-31,100.212997,91.457001,94.489998,64.930000,246.229996,682.059998
2025-11-03,100.089996,91.470001,94.529999,64.900002,245.440002,683.340027
2025-11-04,100.139999,91.480003,93.559998,64.339996,241.199997,675.239990
2025-11-05,99.870003,91.489998,94.160004,64.779999,244.679993,677.580017
2025-11-06,100.239998,91.500000,93.949997,64.160004,240.350006,670.309998


In [ ]:
# Sanity check for parquet reload
import pandas as pd

test_reload = pd.read_parquet("../data/benchmark_prices_named.parquet")
print(test_reload.shape)

(2514, 6)
